In [0]:
%sql
CREATE CATALOG IF NOT EXISTS teste_databricks
COMMENT 'Catálogo de teste para estudo do Unity Catalog'

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS teste_databricks.bronze
COMMENT 'Schema da camada bronze'

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS teste_databricks.silver
COMMENT 'Schema da camada silver'

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS teste_databricks.gold
COMMENT 'Schema da camada gold'

In [0]:
%sql
CREATE TABLE teste_databricks.bronze.sales (
  sale_id INT,
  product STRING,
  quantity INT,
  price DOUBLE,
  sale_date DATE
)
USING DELTA
COMMENT 'Tabela de vendas brutas no formato Delta';


In [0]:
%sql
SELECT * FROM teste_databricks.bronze.sales

In [0]:
%sql
INSERT INTO teste_databricks.bronze.sales (sale_id, product, quantity, price, sale_date)
VALUES
  -- Grupo 1: Notebook
  (1, 'Notebook Dell', 1, 5500.00, DATE'2025-10-01'),
  (2, 'Notebook Dell', 2, 5200.00, DATE'2025-10-02'),
  (3, 'Notebook Dell', 1, 5300.00, DATE'2025-10-03'),
  (4, 'Notebook Dell', 3, 5400.00, DATE'2025-10-04'),
  (5, 'Notebook Dell', 1, 5600.00, DATE'2025-10-05'),

  -- Grupo 2: Monitor
  (6, 'Monitor LG 27"', 2, 1200.00, DATE'2025-10-06'),
  (7, 'Monitor LG 27"', 1, 1150.00, DATE'2025-10-07'),
  (8, 'Monitor LG 27"', 3, 1180.00, DATE'2025-10-08'),
  (9, 'Monitor LG 27"', 2, 1220.00, DATE'2025-10-09'),
  (10, 'Monitor LG 27"', 1, 1250.00, DATE'2025-10-10'),

  -- Grupo 3: Headset
  (11, 'Headset HyperX', 2, 600.00, DATE'2025-10-11'),
  (12, 'Headset HyperX', 1, 590.00, DATE'2025-10-12'),
  (13, 'Headset HyperX', 3, 620.00, DATE'2025-10-13'),
  (14, 'Headset HyperX', 2, 610.00, DATE'2025-10-14'),
  (15, 'Headset HyperX', 1, 630.00, DATE'2025-10-15');


In [0]:
%sql
CREATE OR REPLACE VIEW teste_databricks.bronze.vw_sales_summary AS
SELECT
  product,
  SUM(quantity) AS total_sold,
  ROUND(SUM(quantity * price), 2) AS revenue
FROM teste_databricks.bronze.sales
GROUP BY product;


In [0]:
%sql
SELECT * FROM teste_databricks.bronze.vw_sales_summary

In [0]:
%sql
CREATE VOLUME teste_databricks.bronze.raw_files
COMMENT 'Volume para arquivos brutos de ingestão inicial';

In [0]:
%sql
CREATE OR REPLACE FUNCTION teste_databricks.bronze.calc_bonus(salario DOUBLE, percentual DOUBLE)
RETURNS DOUBLE
COMMENT 'Calcula bônus de acordo com o percentual informado'
RETURN salario * (percentual / 100);

In [0]:
%python
# Ler Parquet
df_parquet = spark.read.parquet("/Volumes/teste_databricks/bronze/raw_files/dados.parquet")
df_parquet.createOrReplaceTempView("vw_parquet")

# Ler CSV
df_csv = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/Volumes/teste_databricks/bronze/raw_files/dados.csv")
)
df_csv.createOrReplaceTempView("vw_csv")

# Ler JSON
df_json = spark.read.json("/Volumes/teste_databricks/bronze/raw_files/dados.json")
df_json.createOrReplaceTempView("vw_json")


In [0]:
%sql
USE CATALOG teste_databricks;
USE SCHEMA bronze;

In [0]:
%sql
SELECT
  nome,
  salario,
  calc_bonus(salario, 10) AS bonus_10
FROM vw_parquet;

In [0]:

%sql
DESCRIBE HISTORY teste_databricks.bronze.sales;


In [0]:

%sql
SELECT count(*) FROM teste_databricks.bronze.sales


In [0]:
%sql
RESTORE TABLE teste_databricks.bronze.sales TO VERSION AS OF 1;